<a href="https://colab.research.google.com/github/yuliiabosher/Adam_Smith_project/blob/main/Congressional_record.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import requests
import os
from google.colab import output
import pprint
from io import BytesIO
import PyPDF2

In [11]:
key = input('enter API key ')
os.environ['Congress_API_Key'] = key
output.clear()

In [21]:
response = requests.get(f'https://api.congress.gov/v3/congressional-record/?y=2024&m=5&d=21&api_key={key}')
pprint.pprint(response.text)

('{\n'
 '    "Results": {\n'
 '        "IndexStart": 1,\n'
 '        "Issues": [\n'
 '            {\n'
 '                "Congress": "118",\n'
 '                "Id": 27400,\n'
 '                "Issue": "88",\n'
 '                "Links": {\n'
 '                    "Digest": {\n'
 '                        "Label": "Daily Digest",\n'
 '                        "Ordinal": 1,\n'
 '                        "PDF": [\n'
 '                            {\n'
 '                                "Part": "1",\n'
 '                                "Url": '
 '"https://www.congress.gov/118/crec/2024/05/21/170/88/CREC-2024-05-21-dailydigest.pdf"\n'
 '                            }\n'
 '                        ]\n'
 '                    },\n'
 '                    "FullRecord": {\n'
 '                        "Label": "Entire Issue",\n'
 '                        "Ordinal": 5,\n'
 '                        "PDF": [\n'
 '                            {\n'
 '                                "Part": "1",\n'
 '       

In [59]:
link_to_pdf_file = response.json()['Results']['Issues'][0]['Links']['FullRecord']['PDF'][0]['Url']
print(link_to_pdf_file)

https://www.congress.gov/118/crec/2024/05/21/170/88/CREC-2024-05-21.pdf


In [69]:
pdf_file = requests.get(link_to_pdf_file)
pdf_data = BytesIO(pdf_file.content)
pdf_reader = PyPDF2.PdfReader(pdf_data)
number_of_pages = len(pdf_reader.pages)
page = pdf_reader.pages[0]
text = page.extract_text()
print(text)

Congressional RecordUNUMEPLURIBUS
United States
of America PROCEEDINGS AND DE BATES OF THE 118th CONGRESS, SECOND SESSION
bThis symbol represents the time of day during the House proceedings, e.g., b1407 is 2:07 p.m.
Matter set in this typeface indicates words inserted or appended, rather than spoken, by a Member of the House on the floor.
.H3359 Vol. 170 WASHINGTON, TUESDAY, MAY 21, 2024 No. 88 
House of Representatives 
The House met at noon and was 
called to order by the Speaker pro tem-
pore (Mr. K USTOFF ). 
f 
DESIGNATION OF SPEAKER PRO 
TEMPORE 
The SPEAKER pro tempore laid be-
fore the House the following commu-nication from the Speaker: 
WASHINGTON , DC, 
May 21, 2024. 
I hereby appoint the Honorable D AVID 
KUSTOFF to act as Speaker pro tempore on 
this day. 
MIKEJOHNSON , 
Speaker of the House of Representatives. 
f 
MORNING-HOUR DEBATE 
The SPEAKER pro tempore. Pursu-
ant to the order of the House of Janu-ary 9, 2024, the Chair will now recog-nize Members from lists submit